In [ ]:
# Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the dataset into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/bick/womens_clothing_reviews.csv')

In [ ]:
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [ ]:
# Preprocess func
def preprocess(text):
    ret = []
    stop_words = set(stopwords.words('english'))
    for txt in text:
        # Tokenize the text
        tokens = word_tokenize(txt.lower())
        # Remove stop words
        filtered_tokens = [token for token in tokens if token not in stop_words]
        # Add filtered tokens to corpus
        ret.append(filtered_tokens)
    return ret

In [ ]:
cnt = 0
for text in df['Review Text']:
    if cnt <= 15:
        print(f"cnt: {cnt}, text: {text}")
    cnt += 1


cnt: 0, text: Absolutely wonderful - silky and sexy and comfortable
cnt: 1, text: Love this dress!  it's sooo pretty.  i happened to find it in a store, and i'm glad i did bc i never would have ordered it online bc it's petite.  i bought a petite and am 5'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.
cnt: 2, text: I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c
cnt: 3, text: I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear i

In [ ]:
df.fillna("N/A", inplace=True)

In [ ]:
df['Review Text'].isna().sum()

0

In [ ]:
for i in range(10):
    print(df['Review Text'][i])

Absolutely wonderful - silky and sexy and comfortable
Love this dress!  it's sooo pretty.  i happened to find it in a store, and i'm glad i did bc i never would have ordered it online bc it's petite.  i bought a petite and am 5'8".  i love the length on me- hits just a little below the knee.  would definitely be a true midi on someone who is truly petite.
I had such high hopes for this dress and really wanted it to work for me. i initially ordered the petite small (my usual size) but i found this to be outrageously small. so small in fact that i could not zip it up! i reordered it in petite medium, which was just ok. overall, the top half was comfortable and fit nicely, but the bottom half had a very tight under layer and several somewhat cheap (net) over layers. imo, a major design flaw was the net over layer sewn directly into the zipper - it c
I love, love, love this jumpsuit. it's fun, flirty, and fabulous! every time i wear it, i get nothing but great compliments!
This shirt is ve

In [ ]:
# Preprocess the text data for word embedding
stop_words = set(stopwords.words('english'))
corpus = []
for text in df['Review Text']:
    # Tokenize the text
    tokens = word_tokenize(text.lower())
    # Remove stop words
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Add filtered tokens to corpus
    corpus.append(filtered_tokens)


In [ ]:
print(len(corpus))

23486


In [ ]:
corpus

In [ ]:
#수정할점. 점, 따옴표, 느낌표같은것도 있음.
# 구두점 없애기. 전처리 더 하기. 
# 수집한 데이터의 양이 너무 적음.
# 제대로 학습했는지 test해보는 방법이 필요함.
# valid, test, 근데 비지도 학습이라서 이런거 할필요 없는거 아닌가?

In [ ]:
# Train a Word2Vec model on the preprocessed corpus
model = Word2Vec(corpus, min_count=1, vector_size=100)

In [ ]:
for i in range(10):
    print(' '.join(corpus[i]), "\n")

absolutely wonderful - silky sexy comfortable 

love dress ! 's sooo pretty . happened find store , 'm glad bc never would ordered online bc 's petite . bought petite 5 ' 8 '' . love length me- hits little knee . would definitely true midi someone truly petite . 

high hopes dress really wanted work . initially ordered petite small ( usual size ) found outrageously small . small fact could zip ! reordered petite medium , ok. overall , top half comfortable fit nicely , bottom half tight layer several somewhat cheap ( net ) layers . imo , major design flaw net layer sewn directly zipper - c 

love , love , love jumpsuit . 's fun , flirty , fabulous ! every time wear , get nothing great compliments ! 

shirt flattering due adjustable front tie . perfect length wear leggings sleeveless pairs well cardigan . love shirt ! ! ! 

love tracy reese dresses , one petite . 5 feet tall usually wear 0p brand . dress pretty package lot dress . skirt long full overwhelmed small frame . stranger altera

In [ ]:
# Compute the similarity between the input sentence and each sentence in the text file
input_sentence = "absolutely wonderful sikly sexy comfortable"
input_tokens = word_tokenize(input_sentence.lower())  # tokenize the input sentence using Okt
similar_sentences = []
for cor in corpus:
    if len(cor) == 0 or len(input_tokens) == 0:
        continue
    similarity = model.wv.n_similarity(input_tokens, cor)
    similar_sentences.append((' '.join(cor), similarity))

# Sort the similar sentences by similarity score in descending order and output the top n sentences
n = 5
top_similar_sentences = sorted(similar_sentences, key=lambda x: x[1], reverse=True)[:n]
for sentence, similarity in top_similar_sentences:
    print("\nsentence: ", sentence, "\nsimilarity: ", similarity)


sentence:  absolutely wonderful - silky sexy comfortable 
similarity:  0.96098536

sentence:  super cute leggings comfortable functional fun design ! 
similarity:  0.86946607

sentence:  top comfortable vas easily dressed ! love 
similarity:  0.867568

sentence:  soft comfortable ! 
similarity:  0.86431676

sentence:  comfy cozy ! 
similarity:  0.86251223
